In [6]:
# Scrape images

import requests
from bs4 import BeautifulSoup

import re

home = "http://www.archiv.birdpictures.de/Kollektionen/pixtacy/index.php?_&-pg=105&-met=vtview&-prt=1"

result = requests.get(home)

if result.status_code == 200:
    soup = BeautifulSoup(result.content, "html.parser")


In [4]:


all_a_tags = soup.findAll('a')
all_a_tags_size = len(all_a_tags)


all_a_tags_with_href = soup.findAll('a', attrs={'href': re.compile("^http://")})
all_a_tags_with_href_size = len(all_a_tags_with_href)


print(f'anchors = {all_a_tags_size}, href anchors = {all_a_tags_with_href_size}')


anchors = 65, href anchors = 64


In [7]:
# Top level links

all_a_tags = soup.findAll('a', attrs={'href': re.compile("^http://")})

reg_exp = re.compile(r'index.php?_$amp;*')

top_links = []
for link in all_a_tags:
    if (len(link.contents) == 3):
        # print(type(link))
        top_links.append(link['href'])

In [8]:
# Downloadable content

downloadable = []

In [9]:
len(top_links)

50

In [10]:
result = requests.get(top_links[0])

# print(top_links[0])

soup = BeautifulSoup(result.content, "html.parser")

images = soup.findAll('img', attrs={'data-preview': re.compile("^http://")})

for image in images:
    print(image.get('data-preview'))


http://www.archiv.birdpictures.de/Kollektionen/pixtacy/images/lores/Abdimstorch/Abdimstorch_35A8864.jpg?
http://www.archiv.birdpictures.de/Kollektionen/pixtacy/images/lores/Abdimstorch/Abdimstorch_35A8888.jpg?
http://www.archiv.birdpictures.de/Kollektionen/pixtacy/images/lores/Abdimstorch/Abdimstorch_35A8910.jpg?
http://www.archiv.birdpictures.de/Kollektionen/pixtacy/images/lores/Abdimstorch/Abdimstorch_35A8933.jpg?


In [11]:
for l in top_links:
    result = requests.get(l)
    if result.status_code == 200:
        # Process images
        soup = BeautifulSoup(result.content, "html.parser")
        images = soup.findAll('img', attrs={'data-preview': re.compile("^http://")})
        for i in images:
            downloadable.append(i)
    else:
        print(f'Failed to fetch results from link: {l}')
        
        
print(len(downloadable))

1111


In [12]:
import urllib.request


s = set()
for l in downloadable:
    
    # print(l.get('data-preview'))
    s.add(l.get('data-pageid'))
    
    
# The number of downloadable items should be the number of "data-pageid"
# Each "data-pageid" is unique and can (therefore) be used as file name
len(s) == len(downloadable)  

True

In [13]:
## Save image files to disk

import os
import validators

path = os.path.join('resources', 'images')

reg_ex = re.compile(' ') 

for l in downloadable:
    
    url = l.get('data-preview')
    
    if validators.url(url):
        file_name =  f"{l.get('data-pageid')}.jpg"
        file_path = os.path.join(path, file_name)
        urllib.request.urlretrieve(url, file_path)

only_files = next(os.walk(path))[2]
print(len(only_files))

998
